In [1]:
from osgeo import gdal
import numpy as np

In [2]:
def calculate_dead_pixel_percentage(band_data):
    total_pixels = np.prod(band_data.shape)
    dead_pixels = np.count_nonzero(band_data == 0)
    return (dead_pixels / total_pixels) * 100

In [3]:
def find_bands_with_dead_pixels(image_path, threshold_percentage=50):
    dataset = gdal.Open(image_path)

    if not dataset:
        print(f"Failed to open the image at {image_path}")
        return

    num_bands = dataset.RasterCount

    bands_with_dead_pixels = []

    for band_num in range(1, num_bands + 1):
        band = dataset.GetRasterBand(band_num)
        band_data = band.ReadAsArray()

        dead_pixel_percentage = calculate_dead_pixel_percentage(band_data)

        if dead_pixel_percentage > threshold_percentage:
            bands_with_dead_pixels.append(band_num)

        band = None  # Close the band to release resources

    dataset = None  # Close the dataset to release resources

    return bands_with_dead_pixels

In [5]:
# Example usage:
image_path = "filtered_non_zero.tif"
threshold_percentage = 50

bands_with_dead_pixels = find_bands_with_dead_pixels(image_path, threshold_percentage)

if bands_with_dead_pixels:
    print(f"Bands with more than {threshold_percentage}% dead pixels: {bands_with_dead_pixels}")
else:
    print(f"No bands found with more than {threshold_percentage}% dead pixels.")

Bands with more than 50% dead pixels: [64, 191, 192, 194]


In [6]:
def filter_bands_by_dead_pixels(input_path, output_path, threshold_percentage=50):
    dataset = gdal.Open(input_path)

    if not dataset:
        print(f"Failed to open the image at {input_path}")
        return

    num_bands = dataset.RasterCount
    selected_bands = []

    for band_num in range(1, num_bands + 1):
        band = dataset.GetRasterBand(band_num)
        band_data = band.ReadAsArray()

        dead_pixel_percentage = calculate_dead_pixel_percentage(band_data)

        if dead_pixel_percentage <= threshold_percentage:
            selected_bands.append(band_num)

    if not selected_bands:
        print(f"No bands found with less than or equal to {threshold_percentage}% dead pixels.")
        return

    # Create a new dataset with selected bands
    driver = gdal.GetDriverByName("GTiff")
    output_dataset = driver.Create(output_path, dataset.RasterXSize, dataset.RasterYSize, len(selected_bands), band.DataType)

    for i, band_num in enumerate(selected_bands, start=1):
        band = dataset.GetRasterBand(band_num)
        band_data = band.ReadAsArray()
        output_dataset.GetRasterBand(i).WriteArray(band_data)

    # Copy geotransform and projection information
    output_dataset.SetGeoTransform(dataset.GetGeoTransform())
    output_dataset.SetProjection(dataset.GetProjection())

    # Close datasets to release resources
    dataset = None
    output_dataset = None


In [7]:
input_image_path = "filtered_non_zero.tif"
output_image_path = "second_filter.tif"
threshold_percentage = 50

filter_bands_by_dead_pixels(input_image_path, output_image_path, threshold_percentage)
print(f"New TIFF file created with bands having less than or equal to {threshold_percentage}% dead pixels: {output_image_path}")

New TIFF file created with bands having less than or equal to 50% dead pixels: second_filter.tif
